<a href="https://colab.research.google.com/github/mimichen123/dgmd-e17/blob/main/object-detection%20/Setup-Data-Collection/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<iframe width="800" height="500"
src="https://www.youtube.com/embed/3jD02dxL6gg"
frameborder="0"
allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture"
allowfullscreen
style="margin: auto; display: block"></iframe>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
DATASET_DIR="/code/object-detection/assets/duckietown_object_detection_dataset"
IMAGE_SIZE = 416
# this is the percentage of real data that will go into the training set (as opposed to the testing set)
REAL_TRAIN_TEST_SPLIT_PERCENTAGE = 0.8

In [ ]:
# Download a dataset of pre-labelled real images.
from utils.misc import runp

# download dataset
if not os.path.exists(DATASET_DIR):
    runp(f"rm -rf {DATASET_DIR}/*")
    runp(f"mkdir -p {DATASET_DIR}/images")
    runp(f"mkdir -p {DATASET_DIR}/labels")
    runp(f"mkdir -p {DATASET_DIR}/train/images")
    runp(f"mkdir -p {DATASET_DIR}/train/labels")
    runp(f"mkdir -p {DATASET_DIR}/val/images")
    runp(f"mkdir -p {DATASET_DIR}/val/labels")
else:
    print("Folder structure already exists!")

# download dataset
if not os.path.exists(f"{DATASET_DIR}/frames"):
    !wget -O /tmp/dataset.zip https://duckietown-public-storage.s3.amazonaws.com/assets/mooc/2022/duckietown_object_detection_dataset.zip
    runp(f"unzip -q /tmp/dataset.zip -d $(dirname {DATASET_DIR})")
    runp(f"rm /tmp/dataset.zip")
else:
    print("Dataset already downloaded!")

ModuleNotFoundError: No module named 'utils'

In [ ]:
#resize images and resize assoicatd bounding boxes
import json
import os
import cv2
import numpy as np
from tqdm import tqdm
from utils.misc import xminyminxmaxymax2xywfnormalized, train_test_split, makedirs, runp

with open(f"{DATASET_DIR}/annotation/final_anns.json") as anns:
    annotations = json.load(anns)

ModuleNotFoundError: No module named 'utils'

In [ ]:
npz_index = 0

all_image_names = []

def save_img(img, boxes, classes):
    global npz_index
    cv2.imwrite(f"{DATASET_DIR}/images/real_{npz_index}.jpg", img)
    with open(f"{DATASET_DIR}/labels/real_{npz_index}.txt", "w") as f:
        for i in range(len(boxes)):
            f.write(f"{classes[i]} "+" ".join(map(str,boxes[i]))+"\n")
    npz_index += 1
    all_image_names.append(f"real_{npz_index}")

filenames = tqdm(os.listdir(f"{DATASET_DIR}/frames"))
for filename in filenames:
    img = cv2.imread(f"{DATASET_DIR}/frames/{filename}")

    orig_y, orig_x = img.shape[0], img.shape[1]
    scale_y, scale_x = IMAGE_SIZE/orig_y, IMAGE_SIZE/orig_x

    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))

    boxes = []
    classes = []

    if filename not in annotations:
        continue

    for detection in annotations[filename]:
        box = detection["bbox"]
        label = detection["cat_name"]

        if label not in ["duckie", "cone"]:
            continue

        orig_x_min, orig_y_min, orig_w, orig_h = box

        x_min = int(np.round(orig_x_min * scale_x))
        y_min = int(np.round(orig_y_min * scale_y))
        x_max = x_min + int(np.round(orig_w * scale_x))
        y_max = y_min + int(np.round(orig_h * scale_y))

        boxes.append([x_min, y_min, x_max, y_max])
        classes.append(1 if label == "duckie" else 2)

    if len(boxes) == 0:
        continue


    boxes = np.array([xminyminxmaxymax2xywfnormalized(box, IMAGE_SIZE) for box in boxes])
    classes = np.array(classes)-1

    save_img(img, boxes, classes)



train_test_split(all_image_names, REAL_TRAIN_TEST_SPLIT_PERCENTAGE, DATASET_DIR)